In [3]:
import ee
import geemap
import pandas as pd
ee.Initialize()


In [8]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
# perdicted images
v5 = ee.Image("projects/pyregence-ee/assets/lucas/tensorflow/vgg16unet_model_jjd_mae_sigmoid_v5_ESTIMATOR_grass_valley").rename('BP_predicted')
vN = ee.Image("projects/pyregence-ee/assets/lucas/tensorflow/BPpredictions_linear_huber_GrassValley").rename('BP_predicted')


# observed 
bp = ee.Image("projects/pyregence-ee/assets/conus/elmfire/production01/bp").rename('BP_observed')

# areas
areas = ee.FeatureCollection("projects/pyregence-ee/assets/lucas/tensorflow/inference_bboxes")
grassvaleyPnt = ee.Geometry.Point([-120.9868, 39.2642])
grassvaley = areas.filterBounds(grassvaleyPnt)
Map.addLayer(areas,{},'areas')
Map.addLayer(grassvaley,{},'gv')

In [124]:
# stack = v5.addBands(bp)
# samples = stack.sample(**{'region':grassvaley,
#                           'scale':30,
#                           'numPixels':2000,
#                           'seed':1337,
#                           'tileScale':6,
#                           'geometries':False})

# stackN = vN.addBands(bp)
# samplesN = stackN.sample(**{'region':grassvaley,
#                           'scale':30,
#                           'numPixels':2000,
#                           'seed':1337,
#                           'tileScale':6,
#                           'geometries':False})

def sample_image(image, region, numPixels, scale=30, seed=1337, geometries=False ):
    samples = image.sample(**{'region':region,
                          'scale':scale,
                          'numPixels':numPixels,
                          'seed':seed,
                          'tileScale':6,
                          'geometries':geometries})
    return samples

def get_samples_df(samples:ee.FeatureCollection, properties:list,):
    # Arrange the sample as a list of lists.
    samp_dict = samples.reduceColumns(ee.Reducer.toList().repeat(len(properties)), properties)
    samp_list = ee.List(samp_dict.get('list'))

    # Save server-side ee.List as a client-side Python list.
    samp_data = samp_list.getInfo()
    df_source = {e : samp_data[i] for i,e in enumerate(properties)}
    source = pd.DataFrame(df_source)
    return source
def default_chart(df,x,y,title,subtitle=""):
    
    chart = alt.Chart(df).mark_circle(size=60).encode(
        y=y,
        x=x,
        opacity=alt.value(0.2),
        color=alt.value("#a83238"),
        ).properties(
            title={'text':title,
                  'subtitle':subtitle}
        ).configure_title(
            fontSize=20,
            font='Roboto',
            align='left',
            color='black',
        )
    return chart

In [128]:
stack_v5 = v5.addBands(bp)
stack_v1 = vN.addBands(bp)

samples_v5 = sample_image(stack_v5, grassvaley, 2500)
samples_v1 = sample_image(stack_v1, grassvaley, 2500)

properties =  ['BP_observed', 'BP_predicted']
df_samples_v5 = get_samples_df(samples_v5, properties)
df_samples_v1 = get_samples_df(samples_v1, properties)

chart_v5 = default_chart(df_samples_v5, x=properties[1], y=properties[0], title="Burn probability observed vs predicted" , subtitle="vgg16unet_model_jjd_mae_sigmoid_v5_ESTIMATOR_grass_valley")
chart_v1 = default_chart(df_samples_v1, x=properties[1], y=properties[0], title="Burn probability observed vs predicted" , subtitle="BPpredictions_linear_huber_GrassValley")


In [129]:
chart_v5

alt.Chart(...)

In [130]:
chart_v1

alt.Chart(...)

In [44]:
# Arrange the sample as a list of lists.
samp_dict = samples.reduceColumns(ee.Reducer.toList().repeat(2), ['BP_observed', 'BP_predicted'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()


In [95]:
import altair as alt
len(samp_data)

source = pd.DataFrame({"BP_observed": samp_data[0], "BP_predicted": samp_data[1]})


chart = alt.Chart(source).mark_circle(size=60).encode(
    y='BP_observed',
    x='BP_predicted',
    opacity=alt.value(0.2),
    color=alt.value("#a83238"),
).properties(
#     title='Burn probibility observed vrs predicted',#ok this works
    title={'text':'Burn probibility observed vrs predicted',
          'subtitle':'vgg16unet_model_jjd_mae_sigmoid_v5_ESTIMATOR_grass_valley'}

).configure_title(
    fontSize=20,
    font='Roboto',
    align='left',
    color='black',
#     fontWeight='lighter'
)
chart

alt.Chart(...)

In [79]:
# chart Kyles runs
# Arrange the sample as a list of lists.
samp_dictN = samplesN.reduceColumns(ee.Reducer.toList().repeat(2), ['BP_observed', 'BP_predicted'])
samp_listN = ee.List(samp_dictN.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_dataN = samp_listN.getInfo()

In [94]:
sourceN = pd.DataFrame({"BP_observed": samp_dataN[0], "BP_predicted": samp_dataN[1]})
# source.head
chartN = alt.Chart(sourceN).mark_circle(size=60).encode(
    y='BP_observed',
    x='BP_predicted',
    opacity=alt.value(0.2),
    color=alt.value("#a83238"),
).properties(
#     title='Burn probibility observed vrs predicted',#ok this works
    title={'text':'Burn probibility observed vrs predicted',
          'subtitle':'BPpredictions_linear_huber_GrassValley'}

).configure_title(
    fontSize=20,
    font='Roboto',
    align='left',
    color='black',
#     fontWeight='lighter'
)

chartN

alt.Chart(...)